<a href="https://colab.research.google.com/github/amotu/COVIDNet/blob/main/CovidNet_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install surfboard
import surfboard
from surfboard.sound import Waveform
from surfboard.feature_extraction import extract_features
import pandas
import numpy as np
import matplotlib.pyplot as plt
import os
from google.colab import drive
import librosa
import librosa.display
import IPython.display as ipd
import tensorflow as tf
from tqdm import tqdm
import sklearn
from sklearn.metrics import classification_report, confusion_matrix
%tensorflow_version 2.x

In [2]:
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [3]:
# Set file/audio locations
os.chdir('/content/gdrive/MyDrive/DSCI400')
new_audio_path = '/content/gdrive/MyDrive/DSCI400/Regroup Audio New/coughvid'
coughnet_audio_path = '/content/gdrive/MyDrive/DSCI400/Coughnet Audio'
coswara_audio_path = '/content/gdrive/MyDrive/DSCI400/Regroup Audio New/cough-coswara'
audio_path = new_audio_path

In [13]:
from utils.Commented.DataGenerator import DataGenerator, Data_Viz, Get_matrix_inputs
from utils.Commented.SurfboardFeatures import GenerateVectors, Reduce_Dim, Get_feature_vectors
from utils.Commented.Modeling import build_model2, build_model, build_cnn_train, build_train
from utils.Commented.Plotter import loss_plot, accuracy_plot, plot_cm
from utils.Commented.Metrics import round_using_t, get_best_threshold, get_predict, get_sen_spec
from utils.Commented.Transfer_Learning import expand_dim, expand_dim_all, make_transfer_model
from utils.Commented import Parameters

# Quick and Easy

In [12]:
from utils.Commented.DenseNet_Model import Transfer_Model
from utils.Commented.CNN_Model import CNN_Model
from utils.Commented.CNN_MLP_Model import CNN_MLP_Model

In [ ]:
test = Transfer_Model(True, True, True, False) # Train the model

In [ ]:
loss_plot(test, ' Transfer ', True)

In [ ]:
accuracy_plot(test, ' Transfer ', True)

In [16]:
test2 = CNN_MLP_Model(False, False, False, False) # Or load weights for the Model

In [19]:
weights = os.path.join(Parameters.checkpoint_loc, 'test')
test2.load_weights(weights)

# Do it youself

# Prep for Modeling


## Get inputs: DataGenerator, Generate Vectors

In [ ]:
# Produces spectrogram images as well as labels for testing, training, and validation sets
x_train, y_train, x_test, y_test, x_val, y_val = Get_matrix_inputs(new_audio_path, coswara_audio_path, coswara_audio_path, Parameters.saved_load_loc, load=True)

In [ ]:
# Generates test, train, and validation feature vectors for the model
test_data_vec, train_data_vec, val_data_vec = Get_feature_vectors(new_audio_path, coswara_audio_path, coswara_audio_path, Parameters.saved_load_loc, load=True)

## DataGenerator Viz

In [ ]:
# Produces image representations of a select number of spectrograms
Data_Viz(5, 5, x_train, y_train)

## Reduce Dim of Vectors

In [ ]:
# Performs dimensionality reduction of the feature vectors
x_test2, x_train2, x_val2 = Reduce_Dim(Parameters.PCA_COMPONENTS, train_data_vec, test_data_vec, val_data_vec)

## Get Input Shapes

In [ ]:
input_shape_1 = x_train.shape[1:]
input_shape_1

In [ ]:
input_shape_2 = x_train2.shape[1:]
input_shape_2

In [ ]:
metric = {1:'sensitivity_specitivity', 2:'precision_recall', 3:'f_score'}

# CNN + MLP


In [ ]:
# Builds a model combining of a Convolutional Neural Network and a Multi-Layer Perceptron to use MFCC data as inputs to differentiate between COVID-19 positive and COVID-19 negative audio clips
new_model = None
new_model = build_model2(input_shape1=input_shape_1, input_shape2=input_shape_2, num_classes=2)

In [ ]:
# Trains CNN + Multi-Layer Perceptron model using input data from MFCCs ------> best_CNN_MLP_weights.h5
model_hist = build_train(new_model, [x_train, x_train2], y_train, Parameters.BATCH_SIZE, Parameters.EPOCHS, Parameters.LEARNING_RATE, check_loc=Parameters.checkpoint_loc, name='test', val_data = ([x_val, x_val2], y_val), show=False)

In [ ]:
predict, threshold, score = get_best_threshold(new_model, [x_test, x_test2], y_test, metric[1], True)
print('Threshold:', threshold, 'Score:', score)

In [ ]:
y_hat = get_predict(predict, threshold)

In [ ]:
tn, fp, fn, tp = plot_cm(y_test.reshape(-1,), y_hat, threshold)
print('true_neg:', tn)
print('false_pos:', fp)
print('false_neg:', fn)
print('true_pos:', tp)

In [ ]:
Sensitivity, Specificity = get_sen_spec(y_test, y_hat)
print('Sensitivity : ', Sensitivity)
print('Specificity : ', Specificity)

In [ ]:
report = classification_report(y_test.reshape(-1,), y_hat, target_names=['0', '1'])
print(report)

In [ ]:
loss_plot(model_hist, ' CNN+SLP ', True)

In [ ]:
accuracy_plot(model_hist, ' CNN+SLP ', True)

# CNN (API)

In [ ]:
# Builds a Convolutional Neural Network (CNN) model 
a = None
a = build_model(input_shape_1, 2)

In [ ]:
# Trains the CNN model on MFCCs
cnn_hist = build_cnn_train(a, x_train, y_train, Parameters.BATCH_SIZE, Parameters.EPOCHS, Parameters.LEARNING_RATE, Parameters.checkpoint_loc, 'test2', False, x_val, y_val)

In [ ]:
predict, threshold, score = get_best_threshold(a, x_test, y_test, metric[1], True)
print('Threshold:', threshold, 'Score:', score)

In [ ]:
y_hat = get_predict(predict, threshold)

In [ ]:
tn, fp, fn, tp = plot_cm(y_test.reshape(-1,), y_hat, threshold)
print('true_neg:', tn)
print('false_pos:', fp)
print('false_neg:', fn)
print('true_pos:', tp)

In [ ]:
report = classification_report(y_test.reshape(-1,), y_hat, target_names=['0', '1'], )
print(report)

In [ ]:
Sensitivity, Specificity = get_sen_spec(y_test, y_hat)
print('Sensitivity : ', Sensitivity)
print('Specificity : ', Specificity)

In [ ]:
loss_plot(cnn_hist, ' CNN ', True)

In [ ]:
accuracy_plot(cnn_hist, ' CNN ', True)

# Transfer Learning

In [ ]:
x_train_3d, x_test_3d, x_val_3d = expand_dim_all(x_train, x_test, x_val)

In [ ]:
models = ['resnet', 'denseNet', 'vgg']

In [ ]:
TransferModel = make_transfer_model((512, 215, 3), models[1])

In [ ]:
transfer_history = build_cnn_train(TransferModel, x_train_3d, y_train, Parameters.BATCH_SIZE, Parameters.EPOCHS, Parameters.LEARNING_RATE, Parameters.checkpoint_loc, 'test3.h5', False, x_val_3d, y_val)

In [ ]:
predict, threshold, score = get_best_threshold(TransferModel, x_test_3d, y_test, metric[1], True)
print('Threshold:', threshold, 'Score:', score)

In [ ]:
y_hat = get_predict(predict, threshold)

In [ ]:
tn, fp, fn, tp = plot_cm(y_test.reshape(-1,), y_hat, threshold)
print('true_neg:', tn)
print('false_pos:', fp)
print('false_neg:', fn)
print('true_pos:', tp)

In [ ]:
report = classification_report(y_test.reshape(-1,), y_hat, target_names=['0', '1'])
print(report)

In [ ]:
Sensitivity, Specificity = get_sen_spec(y_test, y_hat)
print('Sensitivity : ', Sensitivity)
print('Specificity : ', Specificity)

In [ ]:
loss_plot(transfer_history, ' DenseNet ', True)

In [ ]:
accuracy_plot(transfer_history, ' DenseNet ', True)

# Check if best model can Generalize

In [ ]:
x_train_c, y_train_c, x_test_c, y_test_c, x_val_c, y_val_c = Get_matrix_inputs(new_audio_path, coswara_audio_path, new_audio_path, Parameters.saved_load_loc, load=True)
test_data_vec_c, train_data_vec_c, val_data_vec_c = Get_feature_vectors(new_audio_path, coswara_audio_path, new_audio_path, Parameters.saved_load_loc, load=True)
x_test2_c, x_train2_c, x_val2_c = Reduce_Dim(Parameters.PCA_COMPONENTS, train_data_vec, test_data_vec_c, val_data_vec_c)
x_train_3d_c, x_test_3d_c, x_val_3d_c = expand_dim_all(x_train_c, x_test_c, x_val_c)

In [ ]:
predict, threshold, score = get_best_threshold(TransferModel, x_test_3d_c, y_test_c, metric[1], True)
print('Threshold:', threshold, 'Score:', score)

In [ ]:
y_hat = get_predict(predict, threshold)

In [ ]:
tn, fp, fn, tp = plot_cm(y_test_c.reshape(-1,), y_hat, threshold)
print('true_neg:', tn)
print('false_pos:', fp)
print('false_neg:', fn)
print('true_pos:', tp)

In [ ]:
report = classification_report(y_test_c.reshape(-1,), y_hat, target_names=['0', '1'])
print(report)

In [ ]:
Sensitivity, Specificity = get_sen_spec(y_test_c, y_hat)
print('Sensitivity : ', Sensitivity)
print('Specificity : ', Specificity)